# Previsioni delle quantità di CO<sub>2</sub> emesse da veicoli

## Caricamento dei dati
Per questo progetto sono stati utilizzati tre dataset differenti. <br>
Quindi sono necessarie tre esplorazioni separate dei dati e un'omogeneizzazione dei dati, al fine di ottenere un unico dataset per l'addestramento. <br>

In [ ]:
!pip install pykan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, ParameterSampler, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.metrics as metrics
import xgboost as xgb

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from kan import KAN
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
import os.path
from urllib.request import urlretrieve

DATASETS = {
    "car_emissions_spain2022.csv": "https://raw.githubusercontent.com/pietroolivi/dia-datasets/main/car_emissions_spain2022.csv",
    "car_emissions_canada2014.csv": "https://raw.githubusercontent.com/pietroolivi/dia-datasets/main/car_emissions_canada2014/car_emissions_canada2014.csv",
    "car_emissions_uk2013.csv": "https://raw.githubusercontent.com/pietroolivi/dia-datasets/main/car_emissions_uk2013.csv"
}

for filename, dataset in DATASETS.items():
    if not os.path.exists(filename):
      urlretrieve(dataset, filename)

In [ ]:
spain_emissions = pd.read_csv("car_emissions_spain2022.csv")
spain_emissions.head(5)

In [ ]:
canada_emissions = pd.read_csv("car_emissions_canada2014.csv")
canada_emissions.head(5)

In [ ]:
uk_emissions = pd.read_csv("car_emissions_uk2013.csv", low_memory=False)
uk_emissions.head(5)

# **Esplorazione dei singoli dataset**

# Esplorazione del dataset `uk_emissions`

Il dataset in questione è stato reso fruibile dalla Vehicle Certification Agency (VCA) del Department for Transport britannico, al fine di reperire informazioni inerenti a tutte le vetture nuove in vendita nel Regno Unito nel lasso temporale 2000-2013, e quelle usate immatricolate per la prima volta a partire dal 1° marzo 2001. Il termine *nuova* si riferisce ad una qualsiasi automobile allora disponibile per l'acquisto o il leasing presso un concessionario e che non fosse stata precedentemente registrata. Sempre citando la [documentazione](https://www.gov.uk/co2-and-vehicle-tax-tools), abbiamo che entrambi i veicoli nuovi ed usati possono essere cercati per trovarne i corrispondenti:
- Fuel consumption and CO<sub>2</sub> emissions (by make and model)
- vehicle tax information (by make, model, registration date and current tax tables)
- The cost of tax for all vehicle types

Mentre sono disponibili solamente per le macchine nuove le seguenti informazioni:

- Tax band, including Band A (exempt from tax)
- Fuel economy
- Annual fuel running costs
- Company car taxation, based on CO<sub>2</sub> bands
- Alternative fuel types

Potremmo quindi inziare l'analisi del dataset verificando la veridicità di quest'ultimo ragguaglio, ossia controllando che le auto usate assumano valore NaN in corrispondenza delle colonne sopracitate. Sfortunatamente non disponiamo di una variabile che distingua esplicitamente i mezzi nuovi da quelli usati, ma sappiamo che le osservazioni con `date_of_change` (assumendo si riferisca alla data del passaggio di proprietà) diverso da NaN indicano una vettura sicuramente di seconda mano. Quindi sebbene queste rappresentino un sottoinsieme delle usate (mancherebbero quelle usate da un unico proprietario) eseguiamo il test:

In [ ]:
used_cars_subset = uk_emissions.loc[~uk_emissions["date_of_change"].isna()]
columns_supposedly_nan = used_cars_subset[["tax_band",
                                           "standard_12_months",
                                           "standard_6_months",
                                           "first_year_12_months",
                                           "first_year_6_months",
                                           "fuel_cost_12000_miles"]]
print(columns_supposedly_nan)
columns_supposedly_nan.isna().all()

Abbiamo appurato l'attendibilità della documentazione riguardo al valore degli attributi supposti NaN nelle auto usate. Ci rimane da esaminare l'assenza di tipi di carburante alternativi (contraddistinti dal valore _Petrol/E85(Flex Fuel)_) nella feature categorica `fuel_type` in tali osservazioni:

In [ ]:
used_cars_subset["fuel_type"].isin(["Petrol / E85 (Flex Fuel)"]).any()

Possiamo ritenerci soddisfatti. Sarebbe ora lecito chiedersi quali siano i marchi automobilistici più popolari nel dataset. Soddisfiamo la nostra curiosità visualizzando i primi dieci in ordine decrescente per numero di comparse in un diagramma a torta, condensando i restanti brand nello spicchio `Others` per motivi di comprensibilità.

In [ ]:
# n.b., i termini "manufacturer" e "brand" sono utilizzati come sinonimi
print("[" + str(uk_emissions["manufacturer"].nunique()) \
      + "] different manufacturers and ["               \
      + str(uk_emissions["manufacturer"].count())       \
      + "] total cars are registered")
manufacturers = uk_emissions["manufacturer"].value_counts()
majority_brands = manufacturers[:10]
minority_brands = manufacturers[10:]
# oppure raggruppando i brand che rappresentano meno del 5%
# minority_brands_perc = manufacturers[manufacturers < uk_emissions["manufacturer"].count() * 5 / 100]
majority_brands["Others"] = minority_brands.sum()
my_explode = np.append(np.zeros(majority_brands.count() - 1), 0.1)
majority_brands.plot.pie(title = "TOP 10 BRANDS", explode = my_explode, shadow = True, autopct = "%.2f%%", figsize=(6, 6));

In [ ]:
print("OTHERS: 11th to 62nd brand in descending order of popularity:\n")
percentages = minority_brands.to_numpy() * 100 / uk_emissions["manufacturer"].count()
list_of_series = [minority_brands.index.values, minority_brands.values, percentages]
minority_df = pd.DataFrame({"manufacturer"  : minority_brands.index.values,
                            "# occurrences" : minority_brands.values,
                            "percentage"    : percentages})
n_brands = uk_emissions["manufacturer"].nunique()
minority_df.index = range(11, n_brands + 1)
print(minority_df)

Vediamo ora all'interno di ciascuno dei brand graficati in precedenza quale sia il modello con più occorrenze registrate, servendoci di un istogramma dove ci cureremo di mantenere l'ordine dei produttori secondo le percentuali dianzi calcolate, per una maggiore leggibilità.

In [ ]:
brands_and_models = uk_emissions[["manufacturer", "model"]].copy()
brands_and_models = brands_and_models[brands_and_models.manufacturer.isin(majority_brands.index)]
print("Manufacturers left after screening: [" + str(brands_and_models.manufacturer.nunique()) + "]")
majority_brands = majority_brands.drop("Others")
brands_and_models = brands_and_models.value_counts(["manufacturer", "model"]) \
        .reset_index(name = "count")                                          \
        .sort_values(["count"], ascending = False)                            \
        .drop_duplicates(["manufacturer"])                                    \
        .set_index(["manufacturer"])                                          \
        .reindex(majority_brands.index) # ri-ordiniamo secondo la classifica dei brand stilata in precedenza
brands_and_models["car"] = brands_and_models.index + "\n" + brands_and_models["model"] # concateniamo le stringhe
brands_and_models.plot.bar(title = "MOST POPULAR MODEL FOR EACH OF THE TOP 10 BRANDS ", x = "car", y = "count");

Rimarcherei come un marchio automobilistico possa ripetersi più volte nel dataframe per via degli ipotetici diversi modelli ad esso associati, mentre le molteplici apparizioni di uno stesso modello sono giustificate da una o più differenze di valori in variabili attinenti a specifiche tecniche quali la `engine_capacity` o il `fuel_type`. Potremmo infatti trovare, e.g., una _Ford Courier_ sia a benzina (petrol) con cilindrata (engine_capacity) di 1299 cm<sup>3</sup> che a diesel con cilndrata di 1753 cm<sup>3</sup>. Andiamo ad estrarre qualche insight, facendo attenzione a non assumere che non vi possano essere modelli omonimi tra brand diversi (contando quindi i valori unici in coppia con il relativo brand). Di seguito ignoreremo le le differenze di trasmissione.

In [ ]:
n_cars = len(uk_emissions[["manufacturer", "model", "description"]].copy().drop_duplicates())
n_models = len(uk_emissions[["manufacturer", "model"]].copy().drop_duplicates())
n_brands = len(uk_emissions[["manufacturer"]].copy().drop_duplicates())
print("Average number of engine variants per model: [{:.2f}]".format(n_cars / n_models))
print("Average number of models per manufacturer:  [{:.2f}]".format(n_models / n_brands))

Indaghiamo la relazione tra la classi dello [Standard europeo sulle emissioni inquinanti](https://en.wikipedia.org/wiki/European_emission_standards) e lo sprigionamento di CO delle vetture, per mezzo di un diagramma a scatola e baffi. Osserviamo come le medie dei box siano ordinate. La presenza di outliers, assieme al fatto che uno stesso range di valori di monossido di carbonio possa appartenere a più classi, sono chiari indicatori di come il CO non sia, ragionevolmente, l'unico inquinante tenuto in considerazione per determinare l'appartenenza ad una certa categoria `euro_standard`. Com'è infatti possibile verificare nella pagina a cui rinvia l'hyperlink, THC, NMHC, NH<sub>3</sub> e NO<sub>x</sub> sono alcuni tra gli altri composti valutati.

In [ ]:
quartiles_first = uk_emissions.boxplot(column = "co_emissions", by = "euro_standard", showmeans = True);
quartiles_first.set_ylabel("co_emissions");
quartiles_first.set_title("");
plt.ylim(0, 2500);
plt.suptitle("");

Similmente a quanto fatto sopra andiamo, attraverso un diagramma degli estremi e dei quartili, ad analizzare l'eventuale legame tra `tax_band` e `co2_emissions`. Questa volta notiamo tuttavia come il valore massimo di diossido di carbonio (in g/km) di una certa fascia corrisponda al minimo di quella successiva. In effetti, come riportato nella [guida](https://assets.publishing.service.gov.uk/media/6603f64b13397a0011e419be/v149-rates-of-vehicle-tax-for-cars-motorcycles-light-goods-vehicles-and-private-light-goods-vehicles.pdf) redatta dalla Driver and Vehicle Licensing Agency (DVLA), la CO<sub>2</sub> risulterebbe essere, assieme al `fuel_type`, l'unico elemento considerato nel decretare gli scaglioni di tasse.

In [ ]:
quartiles_second = uk_emissions.boxplot(column = "co2", by = "tax_band", showmeans=True, figsize = (10, 16));
quartiles_second.set_ylabel("co2_emissions");
quartiles_second.set_title("");
plt.suptitle("");

Concludiamo l'esplorazione di questo dataset con un grafico a dispersione che leghi l'`engine_capacity` (volume del motore in cm<sup>3</sup>) di ciascuna vettura con i relativi `combined_metric` (consumi in l/100km). Come ci aspettavamo, in linea di massima, al crescere del primo aumenta anche il secondo ed il trend potrebbe, come suggerisce anche l'indice di correlazione di Pearson tra le due colonne, essere approssimato in maniera soddisfacente da una retta.

In [ ]:
uk_emissions.plot.scatter("engine_capacity", "combined_metric", s=7, c="red");
print("Pearson correlation coefficient: [{:.2f}]".format(uk_emissions["engine_capacity"].corr(uk_emissions["combined_metric"])))

# Esplorazione del dataset `spain_emissions`
### Rimozione delle righe riguardanti veicoli elettrici

All'interno del dataset proveniente dal ministero spagnolo sono presenti anche dati riguardanti macchine elettriche. Questa tipologia di dato non è interessante per l'obiettivo del modello, ovvero la quantità di CO<sub>2</sub> emessa.

La cella seguente analizza i valori delle righe che hanno `engine_type` "Eléctricos puros", estraendo solo le colonne sul minimo e massimo di consumo di litri di carburante e sul minimo e massimo di emissioni di CO<sub>2</sub> per km percorso.

In [ ]:
columns = ["emissions_min_gCO2_km", "emissions_max_gCO2_km"]
spain_emissions.loc[spain_emissions["engine_type"] == "Eléctricos puros", columns].isna().all()

Come era previdibile, tutti i valori delle emissioni di CO<sub>2</sub> per km nelle righe riguardanti i veicoli elettrici sono mancanti, quindi le rispettive righe possono essere eliminate, poichè non utili per l'addestramento del modello, il cui obiettivo è la previsione dell'emissione di CO<sub>2</sub> per veicoli a motore termico.

In [ ]:
electric_cars_index = spain_emissions.loc[spain_emissions["engine_type"] == "Eléctricos puros"].index
spain_emissions = spain_emissions.drop(electric_cars_index)
spain_emissions["engine_type"].unique()


Come si può notare dall'output della cella precedente, la colonna `engine_type` non contiene il valore "Eléctricos puros".

### Rimozione delle colonne contenenti solo valori `NaN`
Nel dataset dell'emissioni di CO<sub>2</sub> proveniente dalla Spagna la colonne `type_hybrid`, `electric_consumption_kwh_100km`, `battery_capacity_kwh ` contengono solo valori `NaN`.

In [ ]:
all_nan_columns = ["type_hybrid", "electric_consumption_kwh_100km", "battery_capacity_kwh"]
spain_emissions[all_nan_columns].isna().values.all()

In [ ]:
spain_emissions = spain_emissions.drop(all_nan_columns, axis=1)

### Rimozione delle righe senza un valore valido di trasmissione
La colonna `transmission` contiene il tipo di trasmissione del veicolo, quindi la modalità di cambio della marcia, se automatica o manuale, che nella colonna sono rispettivamente "A" e "M". L'output della prossima cella mostra che sono presenti 35 righe con valori di trasmissioni non validi, poichè nel dataset spagnolo "SC" sta per "Sin clasificasion", ovvero non classificato. Perciò le righe con valori di trasmissione non validi, essendo un numero molto ridotto, verrano eliminate.

In [ ]:
spain_emissions["transmission"].value_counts()

In [ ]:
valid_transmissions = spain_emissions["transmission"].isin(["A", "M"])
spain_emissions["transmission"] =  spain_emissions.loc[valid_transmissions, "transmission"]

# Esplorazione del dataset `canada_emissions`

### Gestione dimensioni motore

All'interno del dataset proveniente dal ministero canadese sono presenti le dimensioni in L (Litri) del motore dei vari veicoli. Decidiamo di trasformarle in cm$^3$ rinominando quindi la feature Engine_cm3.

In [ ]:
canada_emissions = canada_emissions.rename(columns={"Engine size (L)": "Engine_cm3"})
canada_emissions["Engine_cm3"] = canada_emissions["Engine_cm3"] * 1000
canada_emissions.head(5)

### Trattamento valori mancanti

Con la cella seguente verifico se il dataset contiene dei dati mancanti (nan).

In [ ]:
canada_emissions.isna().values.any()

Visto che il risultato è false non c'è alcun valore mancante da gestire.

### Grafici
Questa sezione contiene alcuni grafici che evidenziano alcuni aspetti del dataframe `canada_emissions`.

L'istogramma seguente mostra la distribuzione delle emissioni di CO$_2$ nel dataframe canadese. I valori si concentrano principalmente nell'intervallo [150,350].

In [ ]:
canada_emissions["CO2 emissions (g/km)"].plot.hist(bins=20,title="CO\u2082 emissions distribution");

L'istogramma seguente mostra le dimensioni dei motori dei vari veicoli nel dataframe canadese. I valori si concentrano principalmente nell'intervallo [1500,6000].

In [ ]:
canada_emissions["Engine_cm3"].plot.hist(bins=20,title="Engine size");

Il seguente grafico a torta mostra il numero di cilindri (in percentuale) dei vari veicoli nel dataframe canadese. La maggior parte di essi ha 6 cilindri.

In [ ]:
canada_emissions["Cylinders"].value_counts().plot.pie(title="Cylinders number" , shadow=True, figsize=(6,6), labels=None)
labels = [str(cylinder) + " cylinders" for cylinder in canada_emissions["Cylinders"].value_counts().index]
plt.legend(labels);

Il seguente grafico a barre mostra i vari tipi di veicoli presenti nel dataset canadese con la relativa frequenza.

In [ ]:
canada_vehicle_class = canada_emissions["Vehicle class"].value_counts()
plt.bar(canada_vehicle_class.index, canada_vehicle_class.values)
plt.xticks(rotation='vertical')
plt.title("Vehicle class")
plt.show()

# **Omogeneizzazione ed unione dei dati**
Avendo reperito tre set di dati ciascuno da una fonte differente, sebbene questi concernano il medesimo dominio e condividano pertanto una considerevole quantità di attributi, potremmo essere erroneamente portati a compiere un troncamento delle colonne, conservando esclusivamente quelle appartenenti all'intersezione delle tre tabelle, tuttavia così facendo andremmo a trascurare il caso-limite in cui le features eliminate dovessero essere le uniche legate, attraverso un relazione non randomica, a quella indagata. Una soluzione più scrupolosa consisterebbe, al contrario, nell'**unione** degli **attributi**, colmando opportunamente le celle vuote formatesi. In effetti, come visto a lezione, i valori NAN possono essere sostituiti, tra gli altri, da (a seconda che le colonne siano di tipo categorico o numerico):
- media
- moda
- mediana

Questo ragionamento va ad ogni modo applicato nei limiti del ragionevole: se una variabile dovesse essere presente solamente in uno dei tre dataset, sarebbe piuttosto fuorviante "inventare" i 2/3  dei valori da essa assunti (ipotizzando i dataset di egual dimensione). Decidiamo quindi di mantenere le colonne presenti in almeno due dataset su tre. In aggiunta, le stesse variabili in comune, malgrado facciano riferimento allo stesso concetto, si manifestano saltuariamente sotto denominazioni diverse, ed a loro volta i valori da queste assunti potrebbero avere formato dissimile (e.g. capitalizzazione delle lettere nel caso testuale o numero di cifre significative in quello numerico).







## Omogeneizzazione di `uk_emissions`

Rimuoviamo le colonne proprie di questo dataset, per moderare la presenza di NaN o dati sostitutivi in quello risultante dall'unione:

In [ ]:
uk_emissions = uk_emissions.drop(columns = ["file", "description", "euro_standard", "tax_band", "transmission", "urban_metric",
                                            "extra_urban_metric", "urban_imperial", "extra_urban_imperial", "combined_imperial",
                                            "noise_level", "thc_emissions", "co_emissions", "nox_emissions", "thc_nox_emissions",
                                            "particulates_emissions", "fuel_cost_12000_miles", "fuel_cost_6000_miles", "standard_12_months",
                                            "standard_6_months", "first_year_12_months", "first_year_6_months", "date_of_change"])

Trasformiamo i nomi delle variabili in quelli concordati con gli altri componenti del gruppo:

In [ ]:
uk_emissions.columns = uk_emissions.columns.str.capitalize()
uk_emissions.rename(columns = {"Engine_capacity" : "Engine_cm3",
                               "Combined_metric" : "Fuel_consumption",
                               "Co2" : "CO2_Emissions"}, inplace = True)

Ci troviamo ora dinnanzi ad una delle principali complicazioni indotte dall'utilizzo di più dataset. L'obbiettivo sarebbe quello di manipolare i nomi delle vetture in modo tale da garantire la consistenza semantica del dataset finale: non vorremmo che, e.g., le automobili _BMW 3 Series_ e _Bmw Series 3_ siano considerate differenti, perlomeno sugli attributi `Manufacturer`e `Model`, anche per consentire all'algoritmo di machine learning di trarre conclusioni rispetto al modello specifico. Il problema non si pone per variabili numeriche come i consumi o la cilindrata, dove siamo interessati più al range di appartenenza piuttosto che ai valori precisi. Vista la mancanza di un pattern universale e siccome risulterebbe eccessivamente gravoso indagare tutte le ~90'000 osservazioni, ci limiteremo ad uniformare quantomeno un campione di automobili analizzato.

In [ ]:
uk_emissions.dropna(inplace = True)
uk_emissions["Model"] = uk_emissions["Model"].replace("[\(\[].*?[\)\]]", "", regex = True)
uk_emissions["Model"] = uk_emissions.apply(lambda row : row["Model"].replace(str(row["Manufacturer"]), ''), axis=1)
uk_emissions["Model"] = uk_emissions["Model"].str.split(" ").str[0].str.replace(",", "")
uk_emissions

## Omogeneizzazione di `spain_emissions`

### Traduzione spagnolo-inglese
Il dataset proveniente dalla Spagna contiene due colonne in spagnolo, `engine_type` e `market_segment`. Quindi è necessaria una traduzione dallo spagnolo all'inglese per entrambe le colonne, così da adattarle agli altri due dataset.

Definisco una funzione per sostituire i valori di una colonna di un dataframe con dei nuovi valori passati come argomento, in questo caso la rispettiva traduzione in inglese, tramite una funzione di mapping fornita da pandas.

In [ ]:
def get_column_mapped_values(column, new_values):
    mapping = dict(zip(column.unique(), new_values))
    return column.map(mapping)

In [ ]:
#Traduzione della colonna engine_type
spanish_engine_type = spain_emissions["engine_type"].unique()
english_engine_type = ["Petrol", "Diesel", "Plug-in hybrid", "Petrol hybrid", "Natural gas",
                       "Diesel Hybrid", "Liquefied petroleum gas(LPG)", "Fuel cell", "Extended range"]

spain_emissions["engine_type"] = get_column_mapped_values(spain_emissions["engine_type"], english_engine_type)

Per la traduzione della colonna `market_segment` si fa riferimento al dataset canadese, il quale contiene una colonna per la classificazione dei veicoli, `Vehicle class`.


In [ ]:
canada_emissions["Vehicle class"].unique()

In [ ]:
#Traduzione della colonna market_segment
english_market_segment = ["Minicompact", "Compact", "Small off-road", "Mid-size", "Sport utility vehicle", "Full-size",
                          "Mid-size off-road", "Full-size off-road", "Minivan", "Luxury", "Minivan", "Van: Passenger"] \
                        + (["Van: Cargo"] * 2) + (["Lorry"] * 7)

spain_emissions["market_segment"] = get_column_mapped_values(spain_emissions["market_segment"], english_market_segment)

### Calcolo medie del consumo di carburante e delle emissioni di CO<sub>2</sub>
Sia per il consumo di carburante che per le emissioni di CO<sub>2</sub> sono presenti i valori di minimo, di massimo e la media
WLTP (<i>Worldwide harmonized Light vehicles Test Procedure</i>), ovvero le misure ottenute dal test standard a livello mondiale per il controllo delle emissioni e dei consumi dei veicoli.
Da questi tre valori ne verrà calcolata la media così da avere una sola colonna per i consumi di carburante e una sola colonna per i consumi di CO<sub>2</sub>, la variabile target da predire.

Prima di poter precedere è necessario controllare che in tutte le righe del dataframe sia presente almeno uno dei tre valori richiesti e che sia diverso da 0, sia per calcolare la media del consumo che delle emissioni.
<br>Nella cella successiva si può vedere esattamente in quante righe sono mancanti questi valori necessari.

In [ ]:
def isnaOrIsZero(dataframe):
    return dataframe.isna() | dataframe.eq(0)

In [ ]:
consumption_columns = ["consumption_min_l_100km", "consumption_max_l_100km", "avg_wltp_consumption_l_100km"]
emissions_columns = ["emissions_min_gCO2_km", "emissions_max_gCO2_km", "avg_wltp_emissions_gCO2_km"]

rows_no_consumption = isnaOrIsZero(spain_emissions[consumption_columns]).all(axis=1)
number_rows_missing_consumption = rows_no_consumption.sum()
print("Rows with no consumption_l_100km:", number_rows_missing_consumption)

rows_no_emissions = isnaOrIsZero(spain_emissions[emissions_columns]).all(axis=1)
number_rows_missing_emissions = rows_no_emissions.sum()
print("Rows with no emissions_gCO2_km:", number_rows_missing_emissions)

rows_no_emissions_consumptions = (rows_no_consumption & rows_no_emissions)
number_rows_missing_emissions_consumptions = rows_no_emissions_consumptions.sum()
print("Rows with no emissions_gCO2_km and no consumption_l_km:", number_rows_missing_emissions_consumptions)

Come riportato nell'output della cella precedente, ci sono <strong>1560</strong> righe senza alcun valore di consumo di carburante, <strong>1562</strong> righe senza alcun valore di emissioni di CO<sub>2</sub> e <strong>1560</strong> in cui è assente sia il consumo di carburante che le emissioni di CO<sub>2</sub>. <br>
Nella cella successiva vengono calcolate le medie di carburante ed emissioni di ciascuna riga e vengono inserite in due nuove colonne `consumption_l_100km`, `emissions_gCO2_km`. <br>

In [ ]:
mean_consumption = "consumption_l_100km"
mean_emissions = "emissions_gCO2_km"
spain_emissions[mean_consumption] = spain_emissions[consumption_columns].mean(axis=1)
spain_emissions[mean_emissions] = spain_emissions[emissions_columns].mean(axis=1)

In [ ]:
spain_emissions = spain_emissions.drop(consumption_columns + emissions_columns, axis=1)

Quindi vengono sostituiti i valori mancanti e i valori posti a 0, poichè considerati anch'essi mancanti, siccome le categorie di veicoli considerate dovrebbero presentare dei consumi di carburante e delle emissioni di CO<sub>2</sub> positivi. <br>
La cella successiva mostra le categorie di veicoli in cui sono assenti sia i consumi di carburante che le emissioni. Le due categorie con più occorrenze sono i "Van: Cargo" e "Lorry", due mezzi molto pesanti e con valori differenti rispetto ad altre categorie, quindi sarebbe più corretto riempire i valori assenti o posti a 0, con i mediani delle due rispettive categorie. Mentre per le restanti categorie si può utilizzare direttamente il mediano di tutta la colonna.

In [ ]:
most_missing_market_segments = spain_emissions.loc[rows_no_emissions_consumptions, "market_segment"].value_counts()[:5]
most_missing_market_segments

Le celle successive evidenziano la necessità di usare, come valore per riempire i valori NaN o posti a 0 delle categorie di veicoli "Van: Cargo" e "Lorry", i rispettivi mediani, anzichè usare direttamente il mediano di tutta la colonna, vista l'ampia differenza causata dalla presenza di veicoli di diverse dimensioni e consumi.

In [ ]:
columns = [mean_consumption, mean_emissions]
market_segments = ["Van: Cargo", "Lorry"]

In [ ]:
all_median = pd.DataFrame(spain_emissions[columns].median(), columns = ["All"]).T
van_cargo_lorries = spain_emissions.loc[spain_emissions["market_segment"].isin(market_segments), ["market_segment"] + columns]
van_cargo_lorries_median = van_cargo_lorries[van_cargo_lorries.ne(0)].groupby("market_segment").median()

In [ ]:
pd.concat([all_median, van_cargo_lorries_median]).style.set_caption("Market segments medians")

In [ ]:
def fillnaZeroes(market_segment, columns):
    vehicles = spain_emissions.loc[spain_emissions["market_segment"] == market_segment, columns].replace(0, np.nan)
    vehicles = vehicles.fillna(vehicles.median())
    return vehicles

In [ ]:
for market_segment in market_segments:
    spain_emissions.loc[spain_emissions["market_segment"] == market_segment, columns] = fillnaZeroes(market_segment, columns)

In [ ]:
other_market_segments = set(spain_emissions["market_segment"].unique()) - set(market_segments)
spain_emissions[columns] = spain_emissions[columns].replace(0, np.nan).fillna(spain_emissions[columns].median())

### Utilizzo di valori espliciti per il tipo di trasmissione
Il dataset spagnolo per indicare la tipologia di trasmissione utilizza solamente "A" e "M". Per adattarlo agli altri dataset e per renderlo più chiaro si è deciso di mappare i rispettivi valori con la versione estesa, ovvero "Automatic", "Manual".

In [ ]:
renaming_dict = {"A": "Automatic", "M": "Manual"}
spain_emissions["transmission"] = spain_emissions["transmission"].map(renaming_dict)

### Inserimento dell'anno del modello del veicolo
I dataset canadesi e britannici presentano entrambi una colonna per l'anno del modello(`Model year` in `canada_emissions` e `year` in `uk_emissions`), una feature che può essere importante per la predezioni delle emissioni prodotte. Quindi a partire dagli altri due dataset, viene estratta la colonna dell'anno.

Prima di procedere all'inserimento è necessario un adattamento della colonna `model`, poichè nel dataset `spain_emissions` ciascun valore è formato dal nome della casa automobilistica, ovvero il valore della colonna `make`, dal nome effettivo del modello e da altre informazioni aggiuntive sul modello. Quindi, si procede all'eliminazione della casa automobilistica e delle informazioni aggiuntive per adattarlo alla colonna del modello degli altri due dataset.

La prossime due celle rimuovono suffissi non presenti negli altri due dataset, eliminando le parole contenute nella variabile locale `useless_words`.

In [ ]:
useless_words = {"Canarias", "Vehículos", "Turismos", "Comerciales", "Nuevo", "NUEVO", "Turismos"}
# Per rimuovere suffissi fuorvianti, non presenti negli altri dataset, nei nomi delle case automobilistiche
def remove_useless_suffix(make):
    make_words = make.split()
    make_words = set(make_words) - useless_words
    return " ".join(make_words)

In [ ]:
spain_emissions["make"] = spain_emissions["make"].apply(remove_useless_suffix)

Di seguito, l'adattamento del nome del modello eliminando la casa automobilistica e le informazioni aggiuntive.

In [ ]:
# Per rimuovere il nome della casa automobilistica e le informazioni aggiuntive dal nome del modello
def remove_manufacturer_and_addional_info(make_model):
    make = make_model["make"]
    model = make_model["model"]
    removedManufacturer = model.replace(make, "").split()
    model = [word for word in removedManufacturer if word not in useless_words][0]
    return model

In [ ]:
spain_emissions["model"] = spain_emissions[["make", "model"]].apply(remove_manufacturer_and_addional_info, axis=1)

Dopo l'adattamento del nome della casa automobilistica e del nome del modello del veicolo, si può procedere all'effettivo inserimento dell'anno in base ai valori dell'anno nei dataset `uk_emissions` e `canada_emissions`. I valori dell'anno che risultano non presenti a seguito del join con `uk_emissions` verranno prelevati dal secondo dataset `canada_emissions`. In caso dei valori risultassero ancora mancanti, tali righe verranno scartate dal dataset, poichè prive di un'informazione rilevante ai fini dell'addestramento del modello.

In [ ]:
spain_emissions = (
    pd.merge(
        spain_emissions,
        uk_emissions[["Manufacturer", "Model", "Year"]],
        how = "left",
        left_on = ["make", "model"],
        right_on =  ["Manufacturer", "Model"]
    )
    .drop_duplicates()
)[list(spain_emissions.columns) + ["Year"]]

In [ ]:
spain_emissions_year_na = spain_emissions[spain_emissions["Year"].isna()].drop("Year", axis=1)
spain_emissions_year_canada = (
    pd.merge(
        spain_emissions_year_na,
        canada_emissions[["Make", "Model", "Model year"]],
        how = "left",
        left_on = ["make", "model"],
        right_on =  ["Make", "Model"]
    )
    .drop_duplicates()
    .rename(columns = {"Model year": "Year"})
)[list(spain_emissions.columns)]

La cella a seguire inserisce i valori dell'anno ottenuti dal join con `canada_emissions` al posto dei valori mancanti a seguito del join con `uk_emissions`. Infine, le righe in cui la colonna `year` risulta ancora mancante dopo i due join verranno scartate.   

In [ ]:
spain_emissions["Year"] = spain_emissions["Year"].fillna(spain_emissions_year_canada["Year"])
spain_emissions = spain_emissions.dropna()

### Grafici di `spain_emissions`
Questa sezione contiene alcuni grafici che evidenziano alcuni aspetti del dataframe risultante `spain_emissions`.value_countsunique

L'istogramma seguente mostra la distribuzione delle emissioni di CO<sub>2</sub> nel dataset spagnolo. I valori si concentrano principalmente nell'intervallo [100, 200].

In [ ]:
spain_emissions["emissions_gCO2_km"].plot.hist(bins=20,title="CO\u2082 emissions distribution");

Il seguente grafico mostra la correlazione tra il consumo di carburante e le emissioni di CO<sub>2</sub>. Si evince che esiste una dipendenza tra i due valori, fatta eccezione per alcuni punti.

In [ ]:
spain_emissions.plot.scatter("consumption_l_100km", "emissions_gCO2_km", title="Fuel consumption and CO\u2082 emissions correlation");

Nel grafico a torta è rappresentata la distribuzione delle categorie di veicoli. Una metà è occupata da sole 4 categorie, mentre le altre 8 si spartiscono il restante 50%.

In [ ]:
spain_emissions["market_segment"].value_counts()[:-1].plot.pie(autopct="%.2f%%", title="Market segments distribution", shadow=True, label="", figsize=(10,10));

Nel seguente grafico è mostrata la relazione tra ciascuna categoria di veicoli e le emissioni di CO<sub>2</sub> corrispondenti. Sono presenti molti outlier sia nella parte inferiore che nella parte superiore di ciascun boxplot. Si può notare come i le medie dei valori di veicoli più pesanti e più inquinanti come "Lorry", "Van: Cargo" e "Luxury" siano molto più alti delle medie dei valori corrispondenti ai veicoli più leggeri e meno inquinanti come "Compact", "Minicompact" e "Mid-size". Questa osservazione è vera solo per le medie e i mediani, siccome i valori di ciascun boxplot coprono una buona parte dell'asse y, quindi alcuni veicoli "Compact" hanno valori più alti di alcuni "Lorry".

In [ ]:
spain_emissions.boxplot(column="emissions_gCO2_km", by="market_segment", showmeans=True, figsize=(18, 6)).set_title("CO\u2082 emissions grouped by market segment");

Mentre nel grafico successivo vengono mostrati gli stessi valori, ma in relazione alla tipologia di motore. Quasi tutti boxplot hanno i valori di media e mediana che si assestano tra 150 e 200, presentando però anche in questo caso molti outlier.

In [ ]:
spain_emissions.boxplot(column="emissions_gCO2_km", by="engine_type", showmeans=True, figsize=(18, 6)).set_title("CO\u2082 emissions grouped by market engine type");

### Scelta delle colonne di `spain_emissions` per l'addestramento
Questa ultima parte dell'omogeneizzazione di `spain_emissions` seleziona solo le colonne utilizzate nell'addestramento rinominandole e facendo le ultime modifiche ai valori delle colonne, in modo da utilizzare uno standard comune agli altri dataset.

In [ ]:
old_columns = ["Year", "make", "model", "engine_displacement_cm3", "consumption_l_100km", "engine_type", "emissions_gCO2_km", "transmission"]
new_columns = ["Year", "Manufacturer", "Model", "Engine_cm3", "Fuel_consumption", "Fuel_type", "CO2_Emissions", "Transmission_type"]
columns_dict = dict(zip(old_columns, new_columns))

In [ ]:
spain_emissions = spain_emissions[old_columns].rename(columns = columns_dict)

Da questo punto in poi, Liquefied Petroleum Gas (LPG) verrà semplicemente sostituito con LPG.

In [ ]:
spain_emissions["Fuel_type"] = spain_emissions["Fuel_type"].replace("Liquefied petroleum gas(LPG)", "LPG")

# **Omogeneizzazione di `canada_emissions`**

### Gestione tipi di carburante

All'interno del dataset proveniente dal ministero canadese sono presenti dati riguardanti il tipo di carburante (colonna "Fuel type").

La cella seguente mostra il numero di occorrenze di ogni lettera nella colonna Fuel type.

In [ ]:
canada_emissions["Fuel type"].value_counts()

Riportiamo sotto il significato delle lettere.

- `X`: Benzina normale (ha un indice di ottano più basso ed è quindi meno resistente alla detonazione)
- `Z`: Benzina premium (ha un indice di ottano più alto e offre una maggiore resistenza alla detonazione. È ideale per motori ad alte prestazioni, che hanno un rapporto di compressione                           più elevato e richiedono un carburante più stabile.)
- `D`: Diesel
- `E`: Etanolo(E85)
- `N`: gas naturale

Per motivi di compatibilità dei dataset è superfluo mantenere la distinzione tra benzina normale e benzina premium. Compattiamo quindi le due lettere X e Z in una parola unica: Petrol.
Inoltre trasformiamo anche E in Etanolo, N in Natural gas e D in Diesel per gli stessi motivi di confrontabilità.
Prima di iniziare a modificare il dataframe per poter effettuare tutte le doverose modifiche per renderlo confrontabile con gli altri dataset ne salviamo una copia per evitare di perdere quello originale che potrebbe tornare utile.

In [ ]:
canada_emissions_copy = canada_emissions.copy()
canada_emissions["Fuel type"] = canada_emissions["Fuel type"].replace({'X': 'Petrol', 'Z': 'Petrol', 'D': 'Diesel', 'N': 'Natural Gas', 'E': 'Ethanol'})
canada_emissions["Fuel type"].value_counts()

Il seguente grafico a torta mostra la percentuale dei diversi tipi di carburante presi in considerazione. Si può notare che la maggior parte dei dati riguardano veicoli a benzina.

In [ ]:
fuel_type_colors = {"Petrol": "yellow", "Ethanol": "red", "Diesel": "green", "Natural Gas": "blue"}
canada_fuel_type = canada_emissions["Fuel type"].value_counts()
canada_fuel_type.plot.pie(colors = canada_fuel_type.index.map(fuel_type_colors), shadow=True, figsize=(8,8))
plt.show()

### Eliminazione colonne superflue e rename delle feature importanti

E' inoltre doveroso eliminare le feature City, Highway, Combined che indicano rispettivamente il consumo di carburante: nelle strade di città, nelle autostrade e in entrambe. Eliminiamo inoltre la colonna superflua dell'id, quella del tipo veicolo (Vehicle class) e quella del numero di cilindri (Cylinders) del veicolo, non presente negli altri dataframe.

In [ ]:
canada_emissions = canada_emissions.drop(["_id","Highway (L/100 km)","City (L/100 km)", "Combined (L/100 km)", "Vehicle class", "Cylinders"], axis=1)
canada_emissions.head(5)

La celle seguente serve solo per modificare i nomi delle feature in modo che siano compatibili con gli altri dataset.

In [ ]:
canada_emissions = canada_emissions.rename(columns={"Model year": "Year","Make": "Manufacturer","Fuel type": "Fuel_type", "Transmission": "Transmission_type","Combined (mpg)": "Fuel_consumption", "CO2 emissions (g/km)": "CO2_Emissions"})
canada_emissions.head(5)

### Gestione del tipo di trasmissione

Le prossime celle servono per rendere confrontabili con gli altri dataframe i valori riguardanti il tipo di trasmissione. In particolare trasformiamo tutti i valori di trasmissioni inizianti per A in Automatic e tutti quelli inizianti per M in Manual eccetto per i valori AM (che sarebbero automated manual) che verranno eliminati per evitare incomprensioni dato che gli altri dataset distinguono solamente tra cambio manuale e automatico.

In [ ]:
index_to_drop = canada_emissions[canada_emissions["Transmission_type"].str.startswith("AM")].index
canada_emissions = canada_emissions.drop(index_to_drop)
canada_emissions["Transmission_type"] = canada_emissions["Transmission_type"].replace({r'^M.*': 'Manual',r'^A.*': 'Automatic' }, regex=True)
canada_emissions["Transmission_type"].value_counts()

Mostriamo ora i risultati appena ottenuti in un grafico a torta.

In [ ]:
canada_emissions["Transmission_type"].value_counts().plot.pie(autopct="%.2f%%", shadow=True)
plt.show()

Come ci aspettavamo grazie allì'uso della funzione `value_counts()` i veicoli a cambio automatico sono la maggioranza.

### Dataset risultante

Adattamento finale del nome del modello eliminando la casa automobilistica e le informazioni aggiuntive.

In [ ]:
# Per rimuovere il nome della casa automobilistica e le informazioni aggiuntive dal nome del modello
def remove_manufacturer(make_model):
    make = make_model["Manufacturer"]
    model = make_model["Model"]
    removedManufacturer = model.replace(make, "").split()
    model = [word for word in removedManufacturer if word not in useless_words][0]
    return model

In [ ]:
canada_emissions["Model"] = canada_emissions[["Manufacturer", "Model"]].apply(remove_manufacturer, axis=1)

Mostriamo di seguito il dataframe risultante `canada_emissions`

In [ ]:
canada_emissions.head(10)

# Unione dei dataset

In [ ]:
emissions = pd.concat([spain_emissions, canada_emissions, uk_emissions], ignore_index=True)

Infine, per i contenuti di tipo stringa come `Manufacturer` e `Model`, sarà necessaria una standardizzazione tra i diversi dataset. Lo standard scelto è la maiuscola per il primo carattere e la minuscola per gli altri caratteri.

In [ ]:
capitalize_columns = ["Manufacturer", "Model"]
emissions[capitalize_columns] = emissions[capitalize_columns].apply(lambda col: col.str.capitalize())

Nella prossima cella, mostriamo una breve descrizione in formato tabellare delle feature numeriche del dataset completo

In [ ]:
emissions.describe()

# Addestramento modelli
A seguito dell'esplorazione e dell'omogeneizzazione dei tre dataset, si può procedere all'addestramento dei modelli. I modelli verranno addestrati sulle seguenti feature indipendenti:
- `Year`: anno in cui è stato prodotto il veicolo
- `Manufacturer`: casa automobilistica che ha prodotto il veicolo
- `Model`: nome del modello del veicolo
- `Engine_cm3`: volume del motore in cm<sup>3</sup>
- `Transmission_type`: tipologia di trasmissione del veicolo, ovvero se il cambio della marcia è automatico o manuale
- `Fuel_type`: tipologia di carburante usato nel veicolo. Per esempio può essere benzina, diesel, benzina e ibrida ecc.
- `Fuel_consumption`: i litri di carburante consumati dal veicolo ogni 100 km

La variabile dipendente target dell'addestramento è `CO2_Emissions`, la quantità di CO<sub>2</sub> emessa dal veicolo in grammi per km.

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Data Preparation ----------
X = emissions.drop("CO2_Emissions", axis=1)
y = emissions["CO2_Emissions"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Limit the number of samples for faster testing
X_train = X_train.sample(n=500, random_state=42)
y_train = y_train.loc[X_train.index]
X_test = X_test.sample(n=250, random_state=42)
y_test = y_test.loc[X_test.index]


categorical_features = ['Manufacturer', 'Model', 'Fuel_type', 'Transmission_type']
numerical_features = ['Year', 'Engine_cm3', 'Fuel_consumption']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_features])

def encode_df(df):
    enc = encoder.transform(df[categorical_features])
    enc_df = pd.DataFrame(
        enc, columns=encoder.get_feature_names_out(categorical_features), index=df.index
    )
    return pd.concat([df[numerical_features].reset_index(drop=True),
                      enc_df.reset_index(drop=True)], axis=1)

X_train_enc = encode_df(X_train)
X_test_enc = encode_df(X_test)

def to_tensor(x_df, y_series):
    X_t = torch.tensor(x_df.values, dtype=torch.float32)
    y_t = torch.tensor(y_series.values, dtype=torch.float32).unsqueeze(1)
    return X_t, y_t

X_train_tensor, y_train_tensor = to_tensor(X_train_enc, y_train)
X_test_tensor, y_test_tensor = to_tensor(X_test_enc, y_test)

# Full training dataset
full_dataset = TensorDataset(X_train_tensor, y_train_tensor)

## Definizione dei Modelli: MLP e KAN

In questo blocco vengono definite due architetture di modelli per regressione:

* **`MLP` (Multi-Layer Perceptron)**: una rete neurale feed-forward costruita in modo flessibile a partire da:

  * `input_dim`: numero di feature in input;
  * `hidden_sizes`: lista con il numero di neuroni per ciascun layer nascosto;
  * `dropout`: tasso di dropout per regolarizzare l'addestramento.
    Ogni layer nascosto è seguito da un'attivazione `ReLU` e un livello di `Dropout`. L'output è uno scalare, indicato per problemi di regressione.

* **`build_kan`**: funzione che restituisce un modello **KAN (Kolmogorov–Arnold Networks)**, una rete neurale basata su un'architettura alternativa ai classici MLP, caratterizzata da:

  * una struttura definita tramite la lista `width`,
  * griglie di punti (`grid`) e grado del polinomio (`k`) per le interpolazioni,
  * seme casuale (`seed`) per la riproducibilità e
  * assegnazione del modello al corretto `device` (CPU o GPU).


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_sizes, dropout):
        super().__init__()
        layers = []
        dim = input_dim
        for hs in hidden_sizes:
            layers.append(nn.Linear(dim, hs))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            dim = hs
        layers.append(nn.Linear(dim, 1))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

def build_kan(input_dim, width, grid, k, seed, device):
    return KAN(width=[input_dim] + list(width) + [1], grid=grid, k=k, seed=seed, device=device)

def build_random_forest(**params):
    return RandomForestRegressor(**params)

def build_xgboost(**params):
    return xgb.XGBRegressor(**params)

## Nested Random Search con Early Stopping

Questo blocco di codice implementa una procedura completa di **Nested Random Search** per la selezione di iperparametri di modelli di regressione in PyTorch, integrando una strategia di **Early Stopping** per migliorare l'efficienza del training.

* La classe `EarlyStopper` consente di interrompere l'addestramento anticipatamente se la loss di validazione non migliora per un numero di epoche definito (`patience`), riducendo il rischio di overfitting e velocizzando l'ottimizzazione.
* Le funzioni `train_epoch` ed `eval_loss` gestiscono rispettivamente il training e la valutazione della loss media su un dataset.
* La funzione principale `nested_random_search` esegue una **Nested Cross-Validation**, dove:

  * Il ciclo esterno (outer loop) valuta le prestazioni generali del modello su diversi split train/test.
  * Il ciclo interno (inner loop) esplora combinazioni casuali di iperparametri tramite `ParameterSampler` per ottimizzare la loss di validazione, utilizzando K-Fold CV.
* Per ogni fold esterno, viene selezionata la migliore combinazione di iperparametri trovata all’interno, con successivo riaddestramento sul training set esteso e valutazione finale sul test set.

Il risultato è una lista di tuple contenenti i migliori parametri di modello, parametri di training e loss finale per ciascun fold esterno, utile per valutare la **robustezza e generalizzazione** del modello selezionato.

In [ ]:
class EarlyStopper:
    def __init__(self, patience=3, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def early_stop(self, val_loss):
        # Se la loss migliora (di almeno min_delta), resettiamo il counter
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            # Se la loss non migliora da 'patience' epoche, dobbiamo fermarci
            if self.counter >= self.patience:
                return True
        return False

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def eval_loss(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            total_loss += criterion(model(Xb), yb).item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def nested_random_search_neural(model_builder, param_dist, dataset,
                               outer_folds=5, inner_folds=3, n_iter=10,
                               early_patience=5, early_min_delta=1e-4):
    """
    Esegue Nested Random Search con Randomized Search interno per reti neurali (MLP e KAN)
    Args:
        model_builder: funzione che crea un modello dato model_params
        param_dist: dizionario di liste per ogni iperparametro (model_params + training_params)
        dataset: TensorDataset per il training
        outer_folds, inner_folds: numeri di fold per CV
        n_iter: numero di campioni di Random Search
        early_patience: numero di epoche di pazienza per Early Stopping
        early_min_delta: miglioramento minimo per considerare un progresso
    Returns:
        lista di tuple (best_model_params, best_training_params, test_loss) per ogni outer fold
    """
    # Chiavi riservate per training
    train_keys = ['lr', 'batch_size', 'max_epochs']
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for train_idx, test_idx in outer_cv.split(range(len(dataset))):
        inner_train = Subset(dataset, train_idx)
        inner_test = Subset(dataset, test_idx)
        best_val_loss = float('inf')
        best_model_params, best_train_params = None, None

        # Random Search interno
        for params in ParameterSampler(param_dist, n_iter=n_iter, random_state=42):
            # Separa i parametri del modello da quelli di training
            model_params = {k: v for k, v in params.items() if k not in train_keys}
            train_params = {k: v for k, v in params.items() if k in train_keys}

            # Valutazione su inner_folds con Early Stopping
            inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=42)
            val_losses = []
            for subtrain_idx, val_idx in inner_cv.split(range(len(inner_train))):
                subtrain = Subset(inner_train, subtrain_idx)
                valset = Subset(inner_train, val_idx)
                train_loader = DataLoader(subtrain, batch_size=train_params['batch_size'], shuffle=True)
                val_loader = DataLoader(valset, batch_size=train_params['batch_size'], shuffle=False)

                # Costruisci modello
                model = model_builder(**model_params).to(device)
                optimizer = optim.Adam(model.parameters(), lr=train_params['lr'])
                stopper = EarlyStopper(patience=early_patience, min_delta=early_min_delta)

                # Training
                for epoch in range(train_params['max_epochs']):
                    train_epoch(model, train_loader, optimizer, nn.MSELoss())
                    val_loss = eval_loss(model, val_loader, nn.MSELoss())
                    if stopper.early_stop(val_loss):
                        break

                # Loss di validazione
                val_losses.append(eval_loss(model, val_loader, nn.MSELoss()))

            mean_val = np.mean(val_losses)
            if mean_val < best_val_loss:
                best_val_loss = mean_val
                best_model_params = model_params
                best_train_params = train_params

        # Riaddestramento con i migliori parametri su tutto il sottoinsieme interno con Early Stopping
        full_train_loader = DataLoader(inner_train, batch_size=best_train_params['batch_size'], shuffle=True)
        test_loader = DataLoader(inner_test, batch_size=best_train_params['batch_size'], shuffle=False)
        final_model = model_builder(**best_model_params).to(device)
        optimizer = optim.Adam(final_model.parameters(), lr=best_train_params['lr'])
        stopper = EarlyStopper(patience=early_patience, min_delta=early_min_delta)

        for epoch in range(best_train_params['max_epochs']):
            train_epoch(final_model, full_train_loader, optimizer, nn.MSELoss())
            val_loss = eval_loss(final_model, full_train_loader, nn.MSELoss())
            if stopper.early_stop(val_loss):
                break

        test_loss = eval_loss(final_model, test_loader, nn.MSELoss())
        results.append((best_model_params, best_train_params, test_loss))

    return results

def nested_random_search_sklearn(model_builder, param_dist, X_data, y_data,
                                outer_folds=5, inner_folds=3, n_iter=10):
    """
    Nested Random Search per modelli sklearn e XGBoost
    """
    outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=42)
    results = []

    for train_idx, test_idx in outer_cv.split(X_data):
        X_inner_train, X_inner_test = X_data.iloc[train_idx], X_data.iloc[test_idx]
        y_inner_train, y_inner_test = y_data.iloc[train_idx], y_data.iloc[test_idx]

        best_val_mse = float('inf')
        best_params = None

        for params in ParameterSampler(param_dist, n_iter=n_iter, random_state=42):
            inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=42)
            val_mses = []

            for subtrain_idx, val_idx in inner_cv.split(X_inner_train):
                X_subtrain, X_val = X_inner_train.iloc[subtrain_idx], X_inner_train.iloc[val_idx]
                y_subtrain, y_val = y_inner_train.iloc[subtrain_idx], y_inner_train.iloc[val_idx]

                model = model_builder(**params)
                model.fit(X_subtrain, y_subtrain)
                val_pred = model.predict(X_val)
                val_mse = mean_squared_error(y_val, val_pred)
                val_mses.append(val_mse)

            mean_val_mse = np.mean(val_mses)
            if mean_val_mse < best_val_mse:
                best_val_mse = mean_val_mse
                best_params = params

        # Riaddestramento con i migliori parametri
        final_model = model_builder(**best_params)
        final_model.fit(X_inner_train, y_inner_train)
        test_pred = final_model.predict(X_inner_test)
        test_mse = mean_squared_error(y_inner_test, test_pred)

        results.append((best_params, {}, test_mse))

    return results

## Esecuzione del Nested Random Search su MLP e KAN

In questa sezione viene eseguita la **ricerca di iperparametri tramite Nested Random Search** per due diverse architetture di modelli:
**MLP (Multi-Layer Perceptron)** e **KAN (Kolmogorov–Arnold Networks)**.

### Definizione degli spazi degli iperparametri:

* `mlp_param_dist`: contiene combinazioni di dimensioni dei layer nascosti, tassi di dropout, learning rate, batch size e numero massimo di epoche per il training del modello MLP.
* `kan_param_dist`: definisce i parametri strutturali del modello KAN come larghezza dei layer (`width`), numero di punti griglia (`grid`), grado delle funzioni di base (`k`), e parametri di training.

### Esecuzione:

* Viene eseguita la funzione `nested_random_search` separatamente per ciascun modello.
* I risultati di ogni fold (configurazioni migliori e relativa test loss) vengono stampati a video per permettere un confronto diretto tra le due architetture.

Questo confronto sistematico consente di determinare **quale modello e configurazione di iperparametri offra le migliori prestazioni** su un problema di regressione, valutato con Cross-Validation stratificata e early stopping.


In [ ]:
input_dim = X_train_tensor.shape[1]
mlp_param_dist = {
    'input_dim': [input_dim],
    'hidden_sizes': [(32,32), (64,64), (128,)],
    'dropout': [0.0, 0.2, 0.5],
    'lr': [1e-3, 1e-4],
    'batch_size': [32, 64],
    'max_epochs': [100, 500, 1000]
}

kan_param_dist = {
    'input_dim': [input_dim],
    'width': [(8,4), (16,8)],
    'grid': [5, 10],
    'k': [2, 4],
    'seed': [0],
    'device': [device],
    'lr': [1e-3],
    'batch_size': [32],
    'max_epochs': [100, 500, 1000]
}

rf_param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'random_state': [42]
}

# Parametri per XGBoost
xgb_param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2],
    'random_state': [42]
}

# ---------- Run Nested Random Search ----------
print("=== Nested Random Search Results ===\n")

print("MLP Results:")
mlp_results = nested_random_search_neural(lambda **p: MLP(**p), mlp_param_dist, full_dataset)
for i, (model_p, train_p, loss) in enumerate(mlp_results):
    print(f"Fold {i+1} - Model: {model_p}, Train: {train_p}, Test Loss: {loss:.4f}")

print("\nKAN Results:")
kan_results = nested_random_search_neural(lambda **p: build_kan(**p), kan_param_dist, full_dataset)
for i, (model_p, train_p, loss) in enumerate(kan_results):
    print(f"Fold {i+1} - Model: {model_p}, Train: {train_p}, Test Loss: {loss:.4f}")

print("\nRandom Forest Results:")
rf_results = nested_random_search_sklearn(build_random_forest, rf_param_dist, X_train_enc, y_train)
for i, (model_p, _, loss) in enumerate(rf_results):
    print(f"Fold {i+1} - Params: {model_p}, Test MSE: {loss:.4f}")

print("\nXGBoost Results:")
xgb_results = nested_random_search_sklearn(build_xgboost, xgb_param_dist, X_train_enc, y_train)
for i, (model_p, _, loss) in enumerate(xgb_results):
    print(f"Fold {i+1} - Params: {model_p}, Test MSE: {loss:.4f}")

## Valutazione Finale dei Migliori Modelli: MLP vs KAN

Questa sezione si occupa di confrontare i **migliori modelli trovati** durante la ricerca annidata (`nested_random_search`) per le due architetture:

* Viene definita la funzione `evaluate_model`, che calcola l'**MSE (Mean Squared Error)** di un modello su un `DataLoader`.
* Si identificano i **migliori modelli MLP e KAN**, selezionando quelli con la più bassa *test loss* tra i risultati di cross-validation.
* I modelli vengono **riaddestrati** usando gli stessi dati di test (dell'ultimo fold esterno) per analizzare:

  * **Train loss** ad ogni epoca;
  * **Validation loss** ad ogni epoca.

Questa fase è utile per:

* Verificare se il modello si **adatta bene** al test set senza overfitting;
* Raccogliere dati da **visualizzare graficamente** (es. curva di apprendimento) per analizzare il comportamento delle due architetture nel tempo.


In [ ]:
def evaluate_model_neural(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for Xb, yb in data_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            loss = criterion(model(Xb), yb)
            total_loss += loss.item() * Xb.size(0)
    return total_loss / len(data_loader.dataset)

def count_params(model):
    if hasattr(model, 'parameters'):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        # Per Random Forest e XGBoost, utilizziamo un'approssimazione
        if hasattr(model, 'n_estimators') and hasattr(model, 'max_depth'):
            # Stima approssimativa per Random Forest
            return model.n_estimators * (model.max_depth or 10) * X_train_enc.shape[1]
        else:
            return 0

def get_predictions_neural(model, data_loader):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for Xb, yb in data_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            preds.append(model(Xb).cpu().numpy())
            true.append(yb.cpu().numpy())
    return np.vstack(true), np.vstack(preds)

def compute_metrics_neural(model, data_loader):
    y_true, y_pred = get_predictions_neural(model, data_loader)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    p = count_params(model)
    try:
        r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    except ZeroDivisionError:
        r2_adj = r2
    return mse, r2, r2_adj, p

def compute_metrics_sklearn(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = count_params(model)
    try:
        r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    except ZeroDivisionError:
        r2_adj = r2
    return mse, r2, r2_adj, p

## Valutazione Finale e Confronto tra MLP e KAN

In questa sezione, vengono confrontati i modelli MLP (Multi-Layer Perceptron) e KAN (Kolmogorov–Arnold Networks) utilizzando metriche di regressione e visualizzazioni grafiche per analizzare le loro prestazioni.

### Metriche di Valutazione

Vengono calcolate le seguenti metriche sul test set:

* **MSE (Mean Squared Error)**: misura l'errore quadratico medio tra le predizioni del modello e i valori reali. Un valore più basso indica una migliore accuratezza del modello.

* **R² (Coefficiente di Determinazione)**: indica la proporzione della varianza nei dati dipendenti che è prevedibile dalle variabili indipendenti. Un valore più alto (fino a 1) suggerisce una migliore capacità predittiva.

* **R² Aggiustato**: modifica il R² standard penalizzando l'aggiunta di variabili indipendenti non significative, fornendo una misura più accurata della bontà del modello, specialmente quando si confrontano modelli con un numero diverso di predittori.&#x20;

* **Numero di Parametri Addestrabili**: indica la complessità del modello; modelli con un numero inferiore di parametri sono generalmente preferibili se le prestazioni sono comparabili, poiché tendono a generalizzare meglio e sono meno suscettibili all'overfitting.

### Visualizzazioni

1. **Curve di Loss per Epoca**: grafici che mostrano l'andamento della loss di training e di validazione per ciascun modello nel corso delle epoche, utili per identificare fenomeni di overfitting o underfitting.

2. **Confronto tramite Grafici a Barre**:

   * **Numero di Parametri**: confronta la complessità dei modelli.
   * **R² Aggiustato**: valuta la capacità predittiva tenendo conto della complessità del modello.
   * **MSE di Test**: misura l'accuratezza delle predizioni sui dati di test.([Cross Validated][1])

Queste analisi forniscono una panoramica completa delle prestazioni e della complessità dei modelli MLP e KAN, facilitando la selezione del modello più adatto per il problema in esame.


In [ ]:
# Selezione dei migliori modelli
best_mlp_idx = min(range(len(mlp_results)), key=lambda i: mlp_results[i][2])
best_mlp_params, best_mlp_train, _ = mlp_results[best_mlp_idx]

best_kan_idx = min(range(len(kan_results)), key=lambda i: kan_results[i][2])
best_kan_params, best_kan_train, _ = kan_results[best_kan_idx]

best_rf_idx = min(range(len(rf_results)), key=lambda i: rf_results[i][2])
best_rf_params, _, _ = rf_results[best_rf_idx]

best_xgb_idx = min(range(len(xgb_results)), key=lambda i: xgb_results[i][2])
best_xgb_params, _, _ = xgb_results[best_xgb_idx]

# Creazione dei modelli finali
mlp_model = MLP(**best_mlp_params).to(device)
kan_model = build_kan(**best_kan_params).to(device)
rf_model = build_random_forest(**best_rf_params)
xgb_model = build_xgboost(**best_xgb_params)

# Addestramento dei modelli sklearn
rf_model.fit(X_train_enc, y_train)
xgb_model.fit(X_train_enc, y_train)

# Preparazione del test set per i modelli neurali
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
_, test_idx = list(outer_cv.split(range(len(full_dataset))))[-1]
test_loader = DataLoader(
    Subset(full_dataset, test_idx),
    batch_size=best_mlp_train['batch_size'],
    shuffle=False
)

# Addestramento dei modelli neurali (semplificato per la valutazione finale)
criterion = nn.MSELoss()

# Training MLP
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=best_mlp_train['lr'])
train_loader_mlp = DataLoader(Subset(full_dataset, test_idx), batch_size=best_mlp_train['batch_size'], shuffle=True)
for epoch in range(best_mlp_train['max_epochs']):
    train_epoch(mlp_model, train_loader_mlp, optimizer_mlp, criterion)

# Training KAN
optimizer_kan = optim.Adam(kan_model.parameters(), lr=best_kan_train['lr'])
train_loader_kan = DataLoader(Subset(full_dataset, test_idx), batch_size=best_kan_train['batch_size'], shuffle=True)
for epoch in range(best_kan_train['max_epochs']):
    train_epoch(kan_model, train_loader_kan, optimizer_kan, criterion)

# Calcolo delle metriche
mse_mlp, r2_mlp, r2_adj_mlp, params_mlp = compute_metrics_neural(mlp_model, test_loader)
mse_kan, r2_kan, r2_adj_kan, params_kan = compute_metrics_neural(kan_model, test_loader)
mse_rf, r2_rf, r2_adj_rf, params_rf = compute_metrics_sklearn(rf_model, X_test_enc, y_test)
mse_xgb, r2_xgb, r2_adj_xgb, params_xgb = compute_metrics_sklearn(xgb_model, X_test_enc, y_test)

# Stampa risultati
print("\n=== RISULTATI FINALI ===")
print(f"MLP:    MSE={mse_mlp:.4f}, R²={r2_mlp:.4f}, R²_adj={r2_adj_mlp:.4f}, params={params_mlp}")
print(f"KAN:    MSE={mse_kan:.4f}, R²={r2_kan:.4f}, R²_adj={r2_adj_kan:.4f}, params={params_kan}")
print(f"RF:     MSE={mse_rf:.4f}, R²={r2_rf:.4f}, R²_adj={r2_adj_rf:.4f}, params={params_rf}")
print(f"XGB:    MSE={mse_xgb:.4f}, R²={r2_xgb:.4f}, R²_adj={r2_adj_xgb:.4f}, params={params_xgb}")

# Visualizzazione comparativa
models = ['MLP', 'KAN', 'Random Forest', 'XGBoost']
mse_vals = [mse_mlp, mse_kan, mse_rf, mse_xgb]
r2_vals = [r2_mlp, r2_kan, r2_rf, r2_xgb]
r2_adj_vals = [r2_adj_mlp, r2_adj_kan, r2_adj_rf, r2_adj_xgb]
params_vals = [params_mlp, params_kan, params_rf, params_xgb]

fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# MSE
axs[0, 0].bar(models, mse_vals, color=['blue', 'green', 'orange', 'red'])
axs[0, 0].set_title('MSE Test')
axs[0, 0].set_ylabel('MSE')
axs[0, 0].tick_params(axis='x', rotation=45)

# R²
axs[0, 1].bar(models, r2_vals, color=['blue', 'green', 'orange', 'red'])
axs[0, 1].set_title('R² Score')
axs[0, 1].set_ylabel('R²')
axs[0, 1].tick_params(axis='x', rotation=45)

# R² Aggiustato
axs[1, 0].bar(models, r2_adj_vals, color=['blue', 'green', 'orange', 'red'])
axs[1, 0].set_title('R² Aggiustato')
axs[1, 0].set_ylabel('R² Adjusted')
axs[1, 0].tick_params(axis='x', rotation=45)

# Numero di parametri (scala logaritmica)
axs[1, 1].bar(models, params_vals, color=['blue', 'green', 'orange', 'red'])
axs[1, 1].set_title('Numero di Parametri')
axs[1, 1].set_ylabel('Count')
axs[1, 1].set_yscale('log')
axs[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Tabella riassuntiva
results_df = pd.DataFrame({
    'Model': models,
    'MSE': mse_vals,
    'R²': r2_vals,
    'R²_Adjusted': r2_adj_vals,
    'Parameters': params_vals
})

print("\n=== TABELLA RIASSUNTIVA ===")
print(results_df.round(4))